# Timezones

From version 0.3.0 data is always stored with UTC timestamps.

In [1]:
# packages
import os
import sqlite3
import datetime
import pytz
from emonitor.tools import db_path, history, live

## SQLite database file

In [2]:
# file
name = "fake"
fil = db_path(name)
if os.path.exists(fil):
    print(fil)
else:
    raise Exception("sqlite file not found")

/home/adam/.emonitor/data/fake.db


Queries submitted without timezone information are assumed to be UTC.

In [3]:
# UTC datetime range
start = (2020, 3, 26, 15, 0)
end = (2020, 3, 26, 18, 0)

# data
conn = sqlite3.connect(fil)
df1 = history(conn, start, end)
conn.close()

df1.index

DatetimeIndex(['2020-03-26 15:00:04+00:00', '2020-03-26 15:00:09+00:00',
               '2020-03-26 15:00:14+00:00', '2020-03-26 15:00:19+00:00',
               '2020-03-26 15:00:24+00:00', '2020-03-26 15:00:29+00:00',
               '2020-03-26 15:00:34+00:00', '2020-03-26 15:00:39+00:00',
               '2020-03-26 15:00:44+00:00', '2020-03-26 15:00:49+00:00',
               ...
               '2020-03-26 16:52:17+00:00', '2020-03-26 16:52:22+00:00',
               '2020-03-26 16:52:27+00:00', '2020-03-26 16:52:32+00:00',
               '2020-03-26 16:52:37+00:00', '2020-03-26 16:52:42+00:00',
               '2020-03-26 16:52:47+00:00', '2020-03-26 16:52:52+00:00',
               '2020-03-26 16:52:57+00:00', '2020-03-26 16:53:02+00:00'],
              dtype='datetime64[ns, UTC]', name='TIMESTAMP', length=1342, freq=None)

You can manually specify the timezone of the start / end timestamps. 

In [4]:
end = datetime.datetime.now()
end = end.astimezone(pytz.timezone('CET'))
start = end - datetime.timedelta(minutes=15)

# data
conn = sqlite3.connect(fil)
df2 = history(conn, start, end)
conn.close()

In [5]:
df2.index

DatetimeIndex(['2020-03-26 16:38:08+00:00', '2020-03-26 16:38:13+00:00',
               '2020-03-26 16:38:18+00:00', '2020-03-26 16:38:23+00:00',
               '2020-03-26 16:38:28+00:00', '2020-03-26 16:38:33+00:00',
               '2020-03-26 16:38:38+00:00', '2020-03-26 16:38:43+00:00',
               '2020-03-26 16:38:48+00:00', '2020-03-26 16:38:53+00:00',
               ...
               '2020-03-26 16:52:17+00:00', '2020-03-26 16:52:22+00:00',
               '2020-03-26 16:52:27+00:00', '2020-03-26 16:52:32+00:00',
               '2020-03-26 16:52:37+00:00', '2020-03-26 16:52:42+00:00',
               '2020-03-26 16:52:47+00:00', '2020-03-26 16:52:52+00:00',
               '2020-03-26 16:52:57+00:00', '2020-03-26 16:53:02+00:00'],
              dtype='datetime64[ns, UTC]', name='TIMESTAMP', length=178, freq=None)

However, the it is simpler to explicitly specify the timezone code in the query.  This also adjusts the result.

In [6]:
start = '2020-03-26 15:25:29'
end = '2020-03-26 17:52:29'

# data
conn = sqlite3.connect(fil)
df3 = history(conn, start, end, tz='Europe/Berlin')
conn.close()

In [7]:
df3.index

DatetimeIndex(['2020-03-26 15:25:34+01:00', '2020-03-26 15:25:39+01:00',
               '2020-03-26 15:25:44+01:00', '2020-03-26 15:25:49+01:00',
               '2020-03-26 15:25:54+01:00', '2020-03-26 15:25:59+01:00',
               '2020-03-26 15:26:04+01:00', '2020-03-26 15:26:09+01:00',
               '2020-03-26 15:26:14+01:00', '2020-03-26 15:26:19+01:00',
               ...
               '2020-03-26 17:51:41+01:00', '2020-03-26 17:51:46+01:00',
               '2020-03-26 17:51:52+01:00', '2020-03-26 17:51:57+01:00',
               '2020-03-26 17:52:02+01:00', '2020-03-26 17:52:07+01:00',
               '2020-03-26 17:52:12+01:00', '2020-03-26 17:52:17+01:00',
               '2020-03-26 17:52:22+01:00', '2020-03-26 17:52:27+01:00'],
              dtype='datetime64[ns, Europe/Berlin]', name='TIMESTAMP', length=1745, freq=None)

Some plotting tools force timezone-aware data to UTC (bokeh).  

In this event, use `tz_localize()` to convert timezone adjusted values to timezone-unaware values.

In [8]:
df3.index.tz_localize(None)

DatetimeIndex(['2020-03-26 15:25:34', '2020-03-26 15:25:39',
               '2020-03-26 15:25:44', '2020-03-26 15:25:49',
               '2020-03-26 15:25:54', '2020-03-26 15:25:59',
               '2020-03-26 15:26:04', '2020-03-26 15:26:09',
               '2020-03-26 15:26:14', '2020-03-26 15:26:19',
               ...
               '2020-03-26 17:51:41', '2020-03-26 17:51:46',
               '2020-03-26 17:51:52', '2020-03-26 17:51:57',
               '2020-03-26 17:52:02', '2020-03-26 17:52:07',
               '2020-03-26 17:52:12', '2020-03-26 17:52:17',
               '2020-03-26 17:52:22', '2020-03-26 17:52:27'],
              dtype='datetime64[ns]', name='TIMESTAMP', length=1745, freq=None)